# Requirements

- Python 3.6.2 (NOT python 3.7)
- Sign up for an Intrinio account at https://intrinio.com/ and obtain an API key. Subscription to the 'US Fundamentals and Stock Prices' subscription (free, trial or paid) is required.

Python libaries:
- TensorFlow r1.13
- keras
- requests
- pandas
- matplotlib
- seaborn

# A deep learning price prediction model with TensorFlow

## Download the dataset

### NOTE: Replaced Original Data Loading Section

The original cells for data loading and technical indicator calculation (from the Intrinio API) have been removed and replaced with the single code block below.

**Reason:** The original code relied on the Intrinio API, which requires a paid subscription for technical indicator data, causing errors. This new block uses the free `yfinance` and `pandas-ta` libraries to download the raw price data and calculate the same indicators locally. This is a more modern and robust approach that does not require a paid API key.

In [ ]:
# FIX: This single block replaces the entire original data loading and
# feature engineering section from the notebook.
#
# REASONING:
# 1. Paid API -> Free Alternative: The original code used the Intrinio API,
#    which requires a paid subscription for technical indicators. This new
#    code uses the free yfinance and pandas-ta libraries to achieve the same result.
# 2. Outdated Code: The original code had multiple bugs and used outdated
#    library imports that are not compatible with modern environments like Colab.
# 3. Simplicity: This single, self-contained block is more robust and easier
#    to read and debug than the previous multi-cell approach.

# FIX 1: Install specific, compatible versions of all libraries in the clean environment.
# This creates a stable environment and prevents all version conflicts.
!pip install numpy==1.26.4 pandas==2.2.2 yfinance==0.2.57 pandas-ta==0.3.14b0

# FIX 2: Import all libraries AFTER the correct versions are installed.
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import numpy as np

# --- 1. Define Parameters ---
ticker = 'AAPL'
start_date = '2013-01-01'
end_date = '2018-12-31'

# --- 2. Download Raw Price Data ---
print(f"Downloading OHLCV data for {ticker}...")
prices_df = yf.download(
    ticker,
    start=start_date,
    end=end_date,
    auto_adjust=True,
    progress=False
)

# --- 3. Check if the Download was Successful ---
if prices_df.empty:
    raise ValueError("Data download failed. The DataFrame is empty. Please check your ticker and internet connection.")

# --- 4. Calculate All Technical Indicators ---
print("Calculating technical indicators...")

# Use the pandas-ta extension to directly add all indicators to the DataFrame.
# This is a clean and efficient method.
prices_df.ta.rsi(length=14, append=True)
prices_df.ta.willr(length=14, append=True)
prices_df.ta.ao(fast=5, slow=34, append=True)
prices_df.ta.sma(length=5, append=True)
prices_df.ta.sma(length=15, append=True)
prices_df.ta.sma(length=30, append=True)
prices_df.ta.vwap(append=True)
prices_df.ta.ad(append=True)

# --- 5. Prepare the Final DataFrame for the Model ---
df = prices_df.rename(columns={
    'RSI_14': 'rsi',
    'WILLR_14': 'wr',
    'AO_5_34': 'ao',
    'SMA_5': 'sma_5d',
    'SMA_15': 'sma_15d',
    'SMA_30': 'sma_30d',
    'VWAP_D': 'vwap',
    'AD': 'adtv',
    'Close': 'adj_close_price'
}).dropna()

print("All indicators calculated successfully.")
df.tail()

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


ValueError: Data download failed. The DataFrame is empty. Please check your ticker and internet connection.